设置代理

In [1]:
import os
import requests

# 临时设置环境变量
os.environ['http_proxy'] = 'http://127.0.0.1:7890'
os.environ['https_proxy'] = 'http://127.0.0.1:7890'
os.environ['all_proxy'] = 'socks5://127.0.0.1:7890'

# 测试请求
response = requests.get('https://www.google.com')
print(response.status_code)

200


定义openai的Client

In [2]:
import time

class OpenaiClient:
    def __init__(self, keys=None, start_id=None, proxy=None):
        import openai
        from openai import OpenAI
        
        if isinstance(keys, str):
            keys = [keys]
        if keys is None:
            raise "Please provide OpenAI Key."

        self.key = keys
        self.key_id = start_id or 0
        self.key_id = self.key_id % len(self.key)
        self.api_key = self.key[self.key_id % len(self.key)]
        # 下面这一行base_url="https://api.gpts.vin/v1"是我自己加的
        # self.client = OpenAI(base_url="https://uiuiapi.com/v1", api_key=self.api_key)
        self.client = OpenAI(api_key=self.api_key)

    def chat(self, *args, return_text=False, reduce_length=False, **kwargs):
        while True:
            try:
                completion = self.client.chat.completions.create(*args, **kwargs, timeout=30)
                break
            except Exception as e:
                print(str(e))
                if "This model's maximum context length is" in str(e):
                    print('reduce_length')
                    return 'ERROR::reduce_length'
                time.sleep(0.1)
        if return_text:
            completion = completion.choices[0].message.content
        return completion

    def text(self, *args, return_text=False, reduce_length=False, **kwargs):
        while True:
            try:
                completion = self.client.completions.create(
                    *args, **kwargs
                )
                break
            except Exception as e:
                print(e)
                if "This model's maximum context length is" in str(e):
                    print('reduce_length')
                    return 'ERROR::reduce_length'
                time.sleep(0.1)
        if return_text:
            completion = completion.choices[0].text
        return completion

def run_llm(messages, api_key=None, model_name="gpt-3.5-turbo"):
    if 'gpt' in model_name:
        Client = OpenaiClient
    # elif 'o1' in model_name:
    #     Client = OpenaiClient_o1
    # elif 'claude' in model_name:
    #     Client = ClaudeClient
    # elif 'gemini' in model_name:
    #     Client = GeminiClient
    # elif 'moonshot' in model_name:
    #     Client = KimiClient
    # elif 'deepseek' in model_name:
    #     Client = BailianClient
    # else:
    #     Client = LitellmClient

    agent = Client(api_key)
    response = agent.chat(model=model_name, messages=messages, temperature=0, return_text=True) #temperature used to be 0
    # print(response)
    return response

数据集 导入举例
imdb:25000条
yelp:38000条
sst:2210条  
label:0 negative label:1 positive

In [3]:
from datasets import load_dataset

# Load the dataset
# dataset = load_dataset("./stanfordnlp/imdb")
# dataset = load_dataset("./fancyzhx/yelp")
# dataset = load_dataset("./stanfordnlp/sst")
# test_dataset = dataset["test"]
# test_dataset[12500]["text"]
# print(test_dataset[12500]["label"])

# dataset = load_dataset("./stanfordnlp/imdb", split="test[:100]")  

定义字符级攻击函数

In [4]:
import random

# 定义字符级攻击函数
def text_attack(text, p=0.999, max_alterations=10000000000):
    words = text.split()
    alterations = 0
    attacked_words = []
    
    for word in words:
        if alterations >= max_alterations:
            attacked_words.append(word)
            continue
        
        # Introduce randomness to alter characters within the word
        if len(word) > 1 and random.random() < p:
            word_list = list(word)  # Convert word to list for manipulation
            
            # Randomly delete a character from the word
            if random.random() < 0.5 and len(word_list) > 1:
                idx = random.randrange(len(word_list))
                word_list.pop(idx)
            
            # Randomly change a character (substitution)
            if random.random() < 0.5:
                idx = random.randrange(len(word_list))
                word_list[idx] = random.choice('abcdefghijklmnopqrstuvwxyz')  # Random char substitution
            
            # Randomly swap characters in the word
            if random.random() < 0.5 and len(word_list) > 1:
                idx1, idx2 = random.sample(range(len(word_list)), 2)  # Pick two random indices to swap
                word_list[idx1], word_list[idx2] = word_list[idx2], word_list[idx1]
            
            # Rebuild the word after modification
            new_word = ''.join(word_list)
            attacked_words.append(new_word)
            alterations += 1
        else:
            attacked_words.append(word)
    
    return " ".join(attacked_words)

In [5]:
def create_messages(text):
    messages = [
        {'role': 'system',
         'content': "You are a helpful assistant specialized in sentiment analysis."},
        {'role': 'user',
         'content': f"Please classify the sentiment of the following review:\n\n\"{text}\"\nRespond **only** with 'positive' or 'negative'. Do not include any extra information."}
    ]
    return messages

In [ ]:
from datasets import load_dataset
import openai
openai_key = 'sk-'
model_names = ['gpt-3.5-turbo', 'gpt-4-turbo','gpt-4','gpt-4o']
from tqdm import tqdm
import pickle

import pandas as pd

# 加载数据集
dataset_name = "./stanfordnlp/imdb"
# dataset_name = "./fancyzhx/yelp"
dataset = load_dataset(dataset_name, split="test")
dataset = list(dataset)
dataset = random.sample(dataset, 500)

for model_name in model_names:
    # 用来保存结果的列表
    results = []

    for i in tqdm(range(len(dataset)), desc="Processing", unit="item"):
        item = dataset[i]
        original_text = item["text"]
        # print("原始文本：", original_text)

        # 攻击文本
        attacked_text = text_attack(original_text)
        # print("\n攻击后文本：", attacked_text)

        messages_attacked = create_messages(attacked_text)
        messages_original = create_messages(original_text)

        # 获取原始文本和攻击后文本的情感分析结果
        result_original = run_llm(messages_original, api_key=openai_key, model_name=model_name)
        result_attacked = run_llm(messages_attacked, api_key=openai_key, model_name=model_name)

        # 将结果保存到列表中
        results.append({
            "id": i,
            "label": item["label"],
            "result_original": result_original,
            "result_attacked": result_attacked,
            "original_text": original_text,
            "attacked_text": attacked_text,
        })

    # 保存为 CSV 和 PKL 文件
    df = pd.DataFrame(results)
    df.to_csv(f'{dataset_name}_{model_name}_results.csv', index=False)  # 保存为 CSV

    with open(f'{dataset_name}_{model_name}_results.pkl', 'wb') as f:
        pickle.dump

    df.to_pickle(f'{dataset_name}_{model_name}_results.pkl')  # 保存为 PKL

    print("保存完毕。")

In [ ]:
from datasets import load_dataset
import openai
openai_key = 'sk-'
model_names = ['gpt-3.5-turbo', 'gpt-4-turbo','gpt-4','gpt-4o']
from tqdm import tqdm

import pandas as pd

# 加载数据集
# dataset_name = "./stanfordnlp/imdb"
dataset_name = "./fancyzhx/yelp"
dataset = load_dataset(dataset_name, split="test")
dataset = list(dataset)
dataset = random.sample(dataset, 500)

for model_name in model_names:
    # 用来保存结果的列表
    results = []

    for i in tqdm(range(len(dataset)), desc="Processing", unit="item"):
        item = dataset[i]
        original_text = item["text"]
        # print("原始文本：", original_text)

        # 攻击文本
        attacked_text = text_attack(original_text)
        # print("\n攻击后文本：", attacked_text)

        messages_attacked = create_messages(attacked_text)
        messages_original = create_messages(original_text)

        # 获取原始文本和攻击后文本的情感分析结果
        result_original = run_llm(messages_original, api_key=openai_key, model_name=model_name)
        result_attacked = run_llm(messages_attacked, api_key=openai_key, model_name=model_name)

        # 将结果保存到列表中
        results.append({
            "id": i,
            "label": item["label"],
            "result_original": result_original,
            "result_attacked": result_attacked,
            "original_text": original_text,
            "attacked_text": attacked_text,
        })

    # 保存为 CSV 和 PKL 文件
    df = pd.DataFrame(results)
    df.to_csv(f'{dataset_name}_{model_name}_results.csv', index=False)  # 保存为 CSV
    df.to_pickle(f'{dataset_name}_{model_name}_results.pkl')  # 保存为 PKL

    print("保存完毕。")

直接上sentiment-analysis的过程（不用大模型）

In [ ]:
# from transformers import pipeline

# # 初始化一个情感分析模型
# # Specify the local model path
# model_path = "./model/sentiment-analysis"

# # Initialize the sentiment analysis model with the local path
# classifier = pipeline("sentiment-analysis", model=model_path)

# # 测试其中一条评论
# sample_text = dataset[0]["text"]
# print("原始文本：", sample_text)

# attacked_text = text_attack(sample_text)
# print("\n攻击后文本：", attacked_text)

# # 分别做推理
# result_original = classifier(sample_text)
# result_attacked = classifier(attacked_text)
 
# print("\n原始文本的情感分析结果：", result_original)
# print("攻击后文本的情感分析结果：", result_attacked)

In [6]:
from datasets import load_dataset
import openai
openai_key = 'sk-'
model_names = ['gpt-3.5-turbo', 'gpt-4-turbo','gpt-4o']
from tqdm import tqdm

import pandas as pd

# 加载数据集
# dataset_name = "./stanfordnlp/imdb"
# dataset_name = "./fancyzhx/yelp"
dataset_name = "stanfordnlp/sst-2"
dataset = load_dataset(dataset_name, split="validation")
dataset = list(dataset)
dataset = random.sample(dataset, 500)

for model_name in model_names:
    # 用来保存结果的列表
    results = []

    for i in tqdm(range(len(dataset)), desc="Processing", unit="item"):
        item = dataset[i]
        original_text = item["sentence"]
        # print("原始文本：", original_text)

        # 攻击文本
        attacked_text = text_attack(original_text)
        # print("\n攻击后文本：", attacked_text)

        messages_attacked = create_messages(attacked_text)
        messages_original = create_messages(original_text)

        # 获取原始文本和攻击后文本的情感分析结果
        result_original = run_llm(messages_original, api_key=openai_key, model_name=model_name)
        result_attacked = run_llm(messages_attacked, api_key=openai_key, model_name=model_name)

        # 将结果保存到列表中
        results.append({
            "id": i,
            "label": item["label"],
            "result_original": result_original,
            "result_attacked": result_attacked,
            "original_text": original_text,
            "attacked_text": attacked_text,
        })

    # 保存为 CSV 和 PKL 文件
    df = pd.DataFrame(results)
    df.to_csv(f'{dataset_name}_{model_name}_results.csv', index=False)  # 保存为 CSV
    df.to_pickle(f'{dataset_name}_{model_name}_results.pkl')  # 保存为 PKL

    print("保存完毕。")

Processing: 100%|██████████| 500/500 [32:05<00:00,  3.85s/item]


保存完毕。


Processing: 100%|██████████| 500/500 [33:44<00:00,  4.05s/item]


保存完毕。


Processing: 100%|██████████| 500/500 [31:21<00:00,  3.76s/item]

保存完毕。


In [ ]:
dataset_name = "stanfordnlp/sst-2"
dataset = load_dataset(dataset_name, split="validation")
dataset